In [3]:
import os
import subprocess
import sys

# --- 設定項目 ---

# 1. クローンしたいリポジトリのURLリストが書かれたファイル
url_file_path = 'url_list.txt'

# 2. クローンしたリポジトリを保存するディレクトリ名
clone_to_directory = 'cloned_repositories'


# --- 処理の開始 ---
print(f"--- Starting: Cloning repositories into '{clone_to_directory}' ---")

# 保存先ディレクトリが存在しない場合は作成
os.makedirs(clone_to_directory, exist_ok=True)
print("-" * 50)

# URLリストを読み込む
try:
    with open(url_file_path, 'r') as file:
        # 空行を除外してリスト化
        urls = [line.strip() for line in file.readlines() if line.strip()]
except FileNotFoundError:
    print(f"❌ Error: '{url_file_path}' was not found.")
    print("Please make sure the file exists in the same directory as the notebook.")
    urls = [] # エラーがあった場合はリストを空にする

# 各URLに対してループ処理
for repo_url in urls:
    print(f"Cloning: {repo_url}")
    try:
        # Popenでプロセスを開始し、出力をパイプで受け取る
        # --progressフラグで進捗表示を強制
        process = subprocess.Popen(
            ['git', 'clone', '--progress', repo_url],
            cwd=clone_to_directory,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE, # 進捗はこちらに出力される
            text=True,
            encoding='utf-8',
            errors='replace' # エンコーディングエラーを置換文字で回避
        )

        # 標準エラー(stderr)をリアルタイムで読み込んで進捗を表示
        while process.poll() is None:
            line = process.stderr.readline()
            if line:
                # 行末の改行を削除し、\rでカーソルを先頭に戻して上書き表示
                print(f"   {line.strip()}", end='\r')
        
        # プロセス完了後、進捗表示の行をクリアする
        print(" " * 80, end="\r")

        # プロセスの終了コードを確認
        if process.returncode == 0:
            print("✅ Clone successful.")
        else:
            # エラーが発生した場合、残りのエラー出力を取得して表示
            stdout_err, stderr_err = process.communicate()
            error_message = stderr_err if stderr_err else stdout_err
            print(f"❌ Error cloning. Reason: {error_message.strip()}")

    except FileNotFoundError:
        print("❌ Error: 'git' command not found. Please install Git and ensure it is in your system's PATH.")
        break # Gitがない場合は処理を中断
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        print("-" * 50)

print("Clone process finished.")

--- Starting: Cloning repositories into 'cloned_repositories' ---
--------------------------------------------------
Cloning: https://github.com/MoriwakiYusuke/sbom-python-test.git
✅ Clone successful.                                                             
--------------------------------------------------
Cloning: https://github.com/MoriwakiYusuke/sbom-go-test.git
✅ Clone successful.                                                             
--------------------------------------------------
Clone process finished.


In [4]:
import os
import subprocess

# --- 設定項目 ---

# 1. クローン済みのリポジトリが入っているディレクトリ名 (コード1と合わせる)
clone_to_directory = 'cloned_repositories'

# 2. 各リポジトリ内で実行したいコマンドのリスト
#    実行したいコマンドを、['コマンド', '引数'] の形式で複数記述します.
#    下の例では、①git fetch, ②git status, ③最新ログ1行表示 の3つを順番に実行します.
commands_to_run = [
    ['git', 'fetch'],
    ['git', 'status'],
    ['git', 'log', '-n', '1', '--oneline']
]


# --- 処理の開始 ---
print(f"--- Starting: Executing multiple commands in each repository ---")
print("-" * 40)

# スクリプト実行時の元のパスを保存
original_path = os.getcwd()

try:
    # クローン先のディレクトリに存在するフォルダ(リポジトリ)のリストを取得
    repo_dirs = [d for d in os.listdir(clone_to_directory) if os.path.isdir(os.path.join(clone_to_directory, d))]
    
    if not repo_dirs:
         print("No repositories found to run commands on.")

    # 各リポジトリのディレクトリでループ
    for repo_name in repo_dirs:
        repo_path = os.path.join(clone_to_directory, repo_name)
        print(f"▶️  Entering: {repo_name}")
        
        try:
            # os.chdir() でリポジトリのディレクトリに移動
            os.chdir(repo_path)
            
            # ★★★ 登録されたコマンドを順番に実行するループ ★★★
            for command in commands_to_run:
                print(f"   Executing: {' '.join(command)}")
                
                result = subprocess.run(
                    command,
                    check=True,
                    capture_output=True,
                    text=True
                )
                
                # コマンドの実行結果(標準出力)を表示
                print("   --- Command Output ---")
                print(result.stdout.strip())
                print("   ----------------------")

        except FileNotFoundError:
            # command[0] は実行しようとしたコマンド名 (例: 'git')
            print(f"❌ Error: The command '{command[0]}' was not found.")
            break # コマンドが見つからない場合、以降の処理を中断
        except subprocess.CalledProcessError as e:
            # コマンド実行が失敗した場合
            print(f"❌ Error executing command in {repo_name}.")
            print(f"   Reason: {e.stderr.strip()}")
        finally:
            # ★重要★ 処理の成功・失敗に関わらず、必ず元のディレクトリに戻る
            os.chdir(original_path)
            print("-" * 40)

except FileNotFoundError:
    print(f"❌ Error: The directory '{clone_to_directory}' was not found.")
    print("Please run the cloning script (Code 1) first.")

print("All command execution processes finished.")

--- Starting: Executing multiple commands in each repository ---
----------------------------------------
▶️  Entering: sbom-python-test
   Executing: git fetch
   --- Command Output ---

   ----------------------
   Executing: git status
   --- Command Output ---
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
   ----------------------
   Executing: git log -n 1 --oneline
   --- Command Output ---
1155b7d README更新
   ----------------------
----------------------------------------
▶️  Entering: sbom-go-test
   Executing: git fetch
   --- Command Output ---

   ----------------------
   Executing: git status
   --- Command Output ---
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
   ----------------------
   Executing: git log -n 1 --oneline
   --- Command Output ---
4aac54e README更新
   ----------------------
----------------------------------------
All command execution processes fi